In [1]:
import random
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import os
import keras
import glob
from sklearn.preprocessing import MinMaxScaler

In [2]:
files=glob.glob("Population/*.xlsx",recursive=True)
population=[]
for i in range(0, len(files)):
    parent=pd.read_excel(files[i],header=None).values
    population.append(parent)
population=np.array(population,dtype=object)

In [3]:
all_geometry = []
all_results = []
for i in range(0, population.shape[0]):
    pop = pd.DataFrame(population[i])
    pop = pop.drop(index=[20, 21], axis=0)
    geometry = pop.iloc[14:22, 1:4].values
    results = pop.iloc[82:96, 2:6].values
    all_geometry.append(geometry)
    all_results.append(results)
all_geometry = np.array(all_geometry)
all_results = np.array(all_results)

In [4]:
all_geometry.shape

(38, 8, 3)

In [5]:
def fitness_function(pop,val):
    fitness=[0]*pop.shape[0]

    for i in range(pop.shape[0]):
        
        if (val[i][0][0][0]>=-2 and val[i][0][0][0]<=2) and (val[i][1][0][0]>=-2 and val[i][1][0][0]<=2):
            camber_fitness=abs(val[i][1][0][0]-val[i][0][0][0])/5
            fitness[i]+=camber_fitness
        else:
            fitness[i]=99999
            continue
            
        if (val[i][0][0][1]>=-1 and val[i][0][0][1]<=1) and (val[i][1][0][1]>=-1 and val[i][1][0][1]<=1):
            toe_fitness=abs(val[i][1][0][1]-val[i][0][0][1])/4
            fitness[i]+=toe_fitness
        else:
            fitness[i]=99999
            continue
        
        if (val[i][0][0][2]>=4 and val[i][0][0][2]<=9) and (val[i][1][0][2]>=4 and val[i][1][0][2]<=9):
            fitness[i]+=abs(val[i][1][0][2]-val[i][0][0][2])/2
        else:
            fitness[i]=99999
            continue
            
        if (val[i][0][0][3]>=3 and val[i][0][0][3]<=8) and (val[i][1][0][3]>=3 and val[i][1][0][3]<=8):
            fitness[i]+=abs(val[i][1][0][3]-val[i][0][0][3])/2
        else:
            fitness[i]=99999
            continue

    return fitness

In [6]:
#Use tournament selection

In [7]:
def roulette_selection(population,no_parents,fitness):
    # Roulette_wheel_selection 
    previous_prob=0
    fitness=[1/(x+1e-4) for x in fitness]
    prob_sel=[x/sum(fitness) for x in fitness]
    prob=[]
    parents=np.empty([no_parents,population.shape[1],population.shape[2]])
    for i in range(len(prob_sel)):
        previous_prob=previous_prob+prob_sel[i]
        prob.append(previous_prob)
    
    for i in range(no_parents):
        n=random.random()
        for j in range(len(prob)):
            if n<prob[j]:
                parents[i]=population[j]
                break
        
    return parents

In [8]:
def sus_selection(population,no_parents,fitness):
    pass

In [9]:
def crossover(parent,no_offspring,alpha):
    
    offspring=np.empty([no_offspring,parent.shape[1],parent.shape[2]])
    
    for i in range(no_offspring):
        parent1_index=i%parent.shape[0]
        parent2_index=(i+1)%parent.shape[0]
        n=random.random()
        offspring[i]=n*parent[parent1_index]+(1-n)*parent[parent2_index]
        
    return offspring

In [10]:
def mutation(offspring,mutation_rate):
    
    mutation=np.array(offspring,copy=True)
        
    for k in range(offspring.shape[0]):
        m=random.random()
        if m>mutation_rate:
            continue
            
        for j in range(offspring.shape[1]):
            for i in range(offspring.shape[2]):
                sign=random.randint(0,1)
                if sign==0:
                    mutation[k][j][i]=mutation[k][j][i]+random.random()
                else:
                    mutation[k][j][i]=mutation[k][j][i]-random.random()

    
    return mutation

In [11]:
def prepare_dataset(pop,model):
    X_flat=pop.reshape((pop.shape[0], -1))
    X=[]
    X_final=[]
    bump_val=[200,-60]
        
    for i in range(X_flat.shape[0]):
        for j in range(len(bump_val)):
            X.append(list(np.append(X_flat[i],bump_val[j])))
        #X_final.append(X)
        #X=[]
            
    X_final=np.array(X)
        
    train_df=pd.DataFrame(X_final)
    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_train = scaler.fit_transform(train_df)
    scaled_train_df = pd.DataFrame(scaled_train, columns=train_df.columns.values)
        
    val=[]
    val_total=[]
    X = scaled_train_df[scaled_train_df.columns[:25]].values
    count=0
    for i in range(X.shape[0]):
        count+=1
        val1=model.predict(X[i].reshape(1,X[i].shape[0]))
        val.append(val1)
        if count==2:
            val_total.append(val)
            val=[]
            count=0
    return val_total

In [12]:
model=keras.models.load_model('GA_ANN_normal.h5')

In [13]:
def optimization(pop,no_generation,no_parent,mutation_rate):
    
    for m in range(no_generation):

        val_total=prepare_dataset(pop,model)
        pop_size=38
        no_offspring=pop_size-no_parent
        fitness=fitness_function(pop,val_total)
        parent=roulette_selection(pop,no_parent,fitness)
        offspring=crossover(parent,no_offspring,0.7)
        mutate=mutation(offspring,mutation_rate)
        pop=np.concatenate((pop,mutate),axis=0)
        
    return pop,fitness

In [14]:
pop,fitness=optimization(all_geometry,10,20,0.15)

In [15]:
index=fitness.index(min(fitness))

In [16]:
index

147

In [17]:
fitness[index]

0.05246609449386597

In [18]:
pop[index]

array([[4018.300554743608, -179.84379137428357, 481.91433957545314],
       [4258.9652516804, -179.86656855168275, 470.17913183241245],
       [4081.7817402237415, -463.7585294382277, 255.64778159457353],
       [4053.5486301975407, -185.24739028189856, 636.2128347799335],
       [4312.389777170023, -185.08665834792663, 624.6731411626876],
       [4098.51260162874, -453.6062406980627, 420.8965542074039],
       [4032.899470921729, -489.4269810437343, 311.34758785257975],
       [4046.074853072373, -215.43457180265625, 528.4166002056542]],
      dtype=object)

In [19]:
fitness

[2.037292850017548,
 2.36669340133667,
 99999,
 99999,
 1.8504814863204957,
 2.0061794400215147,
 2.0290383219718935,
 99999,
 99999,
 99999,
 99999,
 2.0947471618652345,
 99999,
 99999,
 99999,
 2.238597106933594,
 99999,
 99999,
 2.1043797016143797,
 0.4934451818466187,
 0.3447071313858032,
 0.35923142433166505,
 0.4101912260055542,
 0.24948527812957763,
 0.583306348323822,
 0.709930145740509,
 99999,
 2.037292850017548,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 0.35100545883178713,
 0.2321349024772644,
 2.192221295833588,
 1.7736334443092345,
 0.3577946424484253,
 0.2574598789215088,
 1.0172239542007446,
 0.30598959922790525,
 0.24948527812957763,
 0.4981070518493652,
 1.8010058522224426,
 0.40359083414077757,
 0.4934451818466187,
 0.4586704850196838,
 0.36102622747421265,
 0.35893008708953855,
 0.2470657467842102,
 0.34909322261810305,
 0.5807246804237366,
 0.8338032126426697,
 0.549214243888855,
 0.26178679466247556,
 0.36883853673934935,
 0.